<a href="https://colab.research.google.com/github/wiebkepeters/NI_PJ_AED/blob/dataloading/SED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 7.2MB/s 


In [2]:
import torch
import torchaudio
from torchaudio import functional as F
from torchaudio import transforms as T

In [3]:
torchaudio.set_audio_backend("sox_io")

**2. MOUNT DRIVE AND CD TO ACCESS DATA (AND FILES IF NOT LOADED FROM REPO))**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Data_Code/

/content/drive/.shortcut-targets-by-id/1nC9alf1XXss0MliDJ7UIwgKnGiUiWn01/Data_Code


**not working yet** *( 1. UPLOAD NEEDED FILES (.py, etc.) AND MIMIC FOLDER STRUCTURE OF REPO (EDIT/ADAPT UPLOADS IF NECESSARY) IF FILES NEED TO BE LOADED FROM REPO (NOT DRIVE) )*

In [5]:
# edit if necessary
directories = ['data_feeders', 'experiments', 'models', 'settings', 'tools']

In [27]:
from google.colab import files

# upload directory by directory 
for dir in directories:
  print('uploading to ' + dir)
  !mkdir $dir
  uploaded = files.upload()
  !mv *.* $dir

uploading to data_feeders


Saving _data_loader_functions.py to _data_loader_functions.py
Saving __init__.py to __init__.py
Saving _tut_sed_synthetic_2016.py to _tut_sed_synthetic_2016.py
uploading to experiments


Saving experiment_process.py to experiment_process.py
Saving __init__.py to __init__.py
Saving _processes.py to _processes.py
uploading to models


Saving baseline_dilated.py to baseline_dilated.py
Saving crnn.py to crnn.py
Saving dessed.py to dessed.py
Saving dessed_dilated.py to dessed_dilated.py
Saving __init__.py to __init__.py
uploading to settings


Saving synthetic_2016_k_33_d_1_1.yaml to synthetic_2016_k_33_d_1_1.yaml
Saving synthetic_2016_k_33_d_10_1.yaml to synthetic_2016_k_33_d_10_1.yaml
Saving synthetic_2016_k_33_d_50_1.yaml to synthetic_2016_k_33_d_50_1.yaml
Saving synthetic_2016_k_33_d_100_1.yaml to synthetic_2016_k_33_d_100_1.yaml
Saving synthetic_2016_k_55_d_1_1.yaml to synthetic_2016_k_55_d_1_1.yaml
Saving synthetic_2016_k_55_d_10_1.yaml to synthetic_2016_k_55_d_10_1.yaml
Saving synthetic_2016_k_55_d_50_1.yaml to synthetic_2016_k_55_d_50_1.yaml
Saving synthetic_2016_k_55_d_100_1.yaml to synthetic_2016_k_55_d_100_1.yaml
Saving synthetic_2016_k_77_d_1_1.yaml to synthetic_2016_k_77_d_1_1.yaml
Saving synthetic_2016_k_77_d_10_1.yaml to synthetic_2016_k_77_d_10_1.yaml
Saving synthetic_2016_k_77_d_10_1_b.yaml to synthetic_2016_k_77_d_10_1_b.yaml
Saving synthetic_2016_k_77_d_50_1.yaml to synthetic_2016_k_77_d_50_1.yaml
Saving synthetic_2016_k_77_d_100_1.yaml to synthetic_2016_k_77_d_100_1.yaml
uploading to tools


Saving file_io.py to file_io.py
Saving __init__.py to __init__.py
Saving metrics.py to metrics.py
Saving preprocess.py to preprocess.py
Saving printing.py to printing.py
Saving various.py to various.py
Saving yaml_loader.py to yaml_loader.py


In [6]:
!ls

conda_dependencies.yml	data_raw     main.py		   README.md
data			experiments  models		   settings
data_feeders		LICENSE      pip_dependencies.txt  tools


In [6]:
# Change directory if needed
%cd drive/MyDrive/Data_Code/

/content/drive/.shortcut-targets-by-id/1nC9alf1XXss0MliDJ7UIwgKnGiUiWn01/Data_Code


**not working yet** *(3. CD TO drive/MyDrive/Data_Code/ (%cd drive/MyDrive/Data_Code/) IF FILES FROM DRIVE ARE USED)*

In [ ]:
# %cd drive/MyDrive/Data_Code/

**not necessary at this point** *( 4. SET PATHS TO DATA )*

In [1]:
# # set paths to data
# train_data = './data/synthetic/training'
# validata_data = './data/synthetic/validate'
# test_data = './data/synthetic/test'

NameError: ignored

**5. CHANGE PARAMS IF NECESSARY (MAKE SURE THAT setting_file CONTAINS CORRECT PATH TO DATA)**

In [5]:
model_to_use = 'baseline'#@param {type:"string"}
config_file = 'synthetic_2016_k_33_d_1_1'#@param {type: "string"}
file_ext = '.yaml'#@param {type: "string"}
settings_dir = './settings'#@param {type:"string"}


**6. START EXPERIMENTS**

In [6]:
from pathlib import Path

from experiments.experiment_process import do_process
from tools.file_io import load_settings_file
from tools.various import get_argument_parser
from tools.printing import InformAboutProcess, date_and_time

In [7]:
with InformAboutProcess('Loading settings'):
        settings = load_settings_file(
            file_name=Path(f'{config_file}{file_ext}'),
            settings_dir=Path(settings_dir))

do_process(
    settings_path=None,
    settings=settings,
    model_to_use=model_to_use)

-- Loading settings... done.

-- Baseline experiment
-- Starting experiment

-- Creating the model... done.
-- Creating training data loader... done.
-- Creating validation data loader... done.
-- Creating optimizer... done.

-- Training examples/batches  :    60 /   30
-- Validation examples/batches:    20 /   10

-- DNN has 3285504 parameters.
-- RNN has 394752 parameters.
-- Classifier has 4112 parameters.
-- Module has 3684368 parameters.

-- Using device: `Tesla T4`.


-- Starting training



RuntimeError: ignored